In [1]:
from core.rabagliati_13_analysis import *
import pandas as pd

<Figure size 432x288 with 0 Axes>

In [2]:
df = pd.read_csv('../data/rabagliati_2013_stimuli.csv')
df = df[~df['target'].isin(['moose', 'mousse'])] #BERT tokenization issues


Formatting the dataframe so we can run BERT through it

In [3]:
df['pos'] = ['n'] * len(df.index)
df['word'] = df['target']
word_pos_pairs = df[['word', 'pos']].drop_duplicates()
word_pos_pairs.to_csv('../data/r_13_for_semcor.csv')

Generating masked language modeling predictions. We replace the target token with a [MASK] and see what BERT predicts when the identity of the word is NOT given.

In [4]:
mask_results = masking_embeds_preds(df)

Inspecting which words have types of both senses that were used in the stimuli vs. only one sense

In [5]:
both_sense_types, one_sense_types, available_sense = check_semcor_data(word_pos_pairs, df)

In [6]:
len(both_sense_types), both_sense_types

(15,
 ['chicken',
  'fish',
  'herbs',
  'letter',
  'button',
  'bow',
  'line',
  'bat',
  'nail',
  'pitcher',
  'band',
  'sun',
  'son',
  'knight',
  'night'])

In [7]:
len(one_sense_types), one_sense_types

(8, ['lamb', 'turkey', 'duck', 'glasses', 'mouse', 'roll', 'cards', 'bark'])

We predict the sense of the masked token by comparing it to SEMCOR data, both using nearest neighbors and picking which sense's centroid is closest to the masked token. 

Global accuracy scores are just above 60%, which shows that even if the token itself is not in the sentence, BERT is able to pick up some sense_related distinctions, but is not as accurate as the participants or the MLE-based algorithm they were compared to.

In [8]:
all_data = generate_sense_predictions(df, mask_results, both_sense_types, one_sense_types, available_sense)
sum(all_data['centroid_preds'] == all_data['wn_sense']) / len(all_data.index)

/Users/sathvik/Desktop/Berkeley/Research/thesis/codebase/core/rabagliati_13_analysis.py:275: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  two_senses['nn_preds'] = e_preds
/Users/sathvik/Desktop/Berkeley/Research/thesis/codebase/core/rabagliati_13_analysis.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  two_senses['centroid_preds'] = c_preds
/Users/sathvik/Desktop/Berkeley/Research/thesis/codebase/core/rabagliati_13_analysis.py:308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

0.6190476190476191

In [9]:
sum(all_data['nn_preds'] == all_data['wn_sense']) / len(all_data.index)

0.6071428571428571

This result shows that unlike 4 year old children in the study, BERT is much more reliably able to distinguish homonyms.

In [12]:
accuracy_table(all_data, 'rel_type')

,rel_type,centroid_acc,nn_acc
0,reg_pol,0.458333,0.500000
1,irreg_pol,0.812500,0.656250
2,hom,0.535714,0.642857


When disambiguating context is in the previous sentence, the accuracy is higher, which is different from the human data- accuracy was higher when the context was in the same sentence.

In [13]:
accuracy_table(all_data, "sentence_ctx")

,sentence_ctx,centroid_acc,nn_acc
0,curr,0.595238,0.595238
1,prev,0.642857,0.619048
